# Wrangle OpenStreetMap Data

The goal is to choose any area of the world in https://www.openstreetmap.org and use data munging techniques to clean the OpenStreetMap data. <br>
<br>
__Map Area__ <br>
- Berkeley, CA, USA: https://mapzen.com/data/metro-extracts/your-extracts/23df9fff1a7c <br>
I'm currently studying in UCB, so I'm more interested to see what database querying reveals about this area. 

## Objectives

- Generate small samples to make auditing easier
- Auditing the tag types, k tag types, and exploring users
- Observing problems when auditing street names and postal codes
- Cleaning dataset
- Converting dataset from XML to CSV format and importing the cleaned .csv files into a SQL database
- Overview statistics of the dataset
- Additional ideas by exploring the data

## Subsetting the Data

Due to the large size of dataset, I started out by looking at a smaller sample first when auditing it to make it easier to iterate on investigation. First, I used k = 100 to make sure that my code works, and then reduced value of k to check for the most common problems to clean. <br>
The complete code for subsetting the data can be found here:

## Auditing Tag Types

In order to get the feeling on how much of which data I can expect to have in this map area, I used the iterative parsing to process the map file and found out all types of tags there and the count for each tag. <br>
The complete code for auditing the tag types can be found here:

In [1]:
def count_tags(filename):
    file = open(filename, "r")
    tags = {}
    for event, elem in ET.iterparse(file):
        if elem.tag in tags.keys():
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags

In [ ]:
{'bounds': 1,
 'member': 7325,
 'nd': 575011,
 'node': 492980,
 'osm': 1,
 'relation': 588,
 'tag': 185978,
 'way': 69973}

After reading through the documentation for OSM XML file format, I found that our data primitives are nodes, ways, and relations. <br>
- a block of nodes containing especially the location in the WGS84 reference system
    - the tags of each node
- a block of ways
    - the references to its nodes for each way
    - the tags of each way
- a block of relations
    - the references to its members for each relation
    - the tags of each relation

## k Tag Types

Before I process the data and add it into my database, I also need to check the "k" value for each "$<tag>$" and see if there are any potential problems. <br>
I used 3 regular expressions to check for certain patterns in the tags, so that I'll have a count of each of four tag categories in a dictionary:
- "lower", for tags that contain only lowercase letters and are valid
- "lower_colon", for otherwise valid tags with a colon in their names
- "problemchars", for tags with problematic characters, and
- "other", for other tags that do not fall into the other three categories.

In [ ]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

Then, found a list of tags that matches above regular expressions. The complete code can be found here: 

In [3]:
def key_type(element, keys):
    if element.tag == "tag":
        val = element.get("k")
        if bool(lower.search(val)):
            keys["lower"] += 1
        elif bool(lower_colon.search(val)):
            keys["lower_colon"] += 1
        elif bool(problemchars.search(val)):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
        pass
        
    return keys

In [ ]:
{'lower': 133257, 'lower_colon': 50572, 'other': 2149, 'problemchars': 0} 

## Exploring Users

I'm also interested on how many unique users have contributed to the map in Berkeley area. <br>
The complete code can be found here:

In [ ]:
def get_user(element):
    if element.get('uid'):
        uid = element.attrib["uid"]
        return uid
    else:
        return None
    
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            users.add(get_user(element))
        pass

    return users

users = process_map('berkeley_california.osm')
pprint.pprint(users)
print(len(users)) 

len(users) = 555

## Auditing Street Names

To clean this dataset, I want to audit the street names in tags and return those unexpected street types to the appropriate ones in the expected list I created. <br>
Notice that, the original expected list was not perfect, so we need to update it when auditing street names. <br>
The complete code can be found here:

In [ ]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Alameda", "Broadway", "Circle", "Freeway", "Hall",
            "Highway", "Loma", "Path", "Plaza", "Steps", "Terrace", "View", "Walk", "Way"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [ ]:
{'10675': set(['10675']),
 '155': set(['University Ave #155']),
 '411': set(['Horton Street Unit #411']),
 'Ave': set(['Shattuck Ave', 'Telegraph Ave']),
 'Blvd': set(['Canal Blvd']),
 'St': set(['2nd St'])} 

Here we can see that the main problem for street names is that some of them are over abbreviated. The format of numbers can be treated as something like apartment number. So, here I choose to ignore them, and change the names that are over abbreviated.

## Auditing Postal Codes

To clean this dataset, I also want to audit the postal codes in tags and return the postal codes that are not presented in appropriate way. <br>
The postal code in Berkeley should be started with 947. Otherwise, it should be considered as inappropriate. <br>
The complete code can be found here:

In [ ]:
def audit_zipcodes(osmfile):
    # iter through all zip codes, collect all the zip codes that does not start with 947
    osm_file = open(osmfile, "r")
    zip_codes = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "addr:postcode" and not tag.attrib['v'].startswith('947'):
                    if tag.attrib['v'] not in zip_codes:
                        zip_codes[tag.attrib['v']] = 1
                    else:
                        zip_codes[tag.attrib['v']] += 1
    return zip_codes

94109 1 <br>
94607 1 <br>
94605 1 <br>
95476 2 <br>
94804 16 <br>
94618 74 <br>
94608 101 <br>
94609 36 <br>
93710 1 <br>
94530 73 <br>
94611 1075 <br>
95430 2

Here we can see that almost every incorrect postal codes are started with 94, that means they are actually very near to Berkeley. <br>
After checking all of those codes, I found that they are basically postal codes for Oakland, Richmond, and San Francisco, which are neighbors of Berkeley.

## Problems Observed

Some of the problems I observed: <br>
- The format of street names is not uniform, because some of them are over abbreviated. I need to clean those later.
- The other problem is some inappropriate postal codes, which actually from the neighbors of Berkeley.

## Cleaning Street Names

Because some street names are over abbreviated, I need to fix them by mapping to the expected list. <br>
The complete code can be found here:

In [ ]:
mapping = { "St": "Street",
            "Ave": "Avenue",
            "Blvd": "Boulevard"
            }
def update_name(name, mapping):
    m = street_type_re.search(name)
    if m.group() not in expected:
        if m.group() in mapping.keys():
            name = re.sub(m.group(), mapping[m.group()], name)
    return name

Canal Blvd => Canal Boulevard <br>
Telegraph Ave => Telegraph Avenue <br>
Shattuck Ave => Shattuck Avenue <br>
2nd St => 2nd Street

## Importing Data Into SQL

After auditing is complete the next step is to prepare the data to be inserted into a SQL database. To do so I need to parse the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to .csv files. These csv files can then easily be imported to a SQL database as tables. <br>
The complete code can be found here: <br>
Defined schema for the .csv files and the eventual tables can be found here:

In [ ]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
                             
SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [ ]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        for child in element:
            node_tags = {}
            if PROBLEMCHARS.match(child.attrib['k']):
                continue
            elif LOWER_COLON.match(child.attrib['k']):
                node_tags['id'] = element.attrib['id']
                node_tags['key'] = child.attrib['k'].split(":", 1)[1]
                node_tags['type'] = child.attrib['k'].split(":", 1)[0]             
                # Use cleaning function:
                if child.attrib['k'] == 'addr:street':
                    node_tags['value'] = update_name(child.attrib['v'], mapping)
                else:
                    node_tags['value'] = child.attrib['v']
            else:
                node_tags['type'] = 'regular'
                node_tags['key'] = child.attrib['k']
                node_tags['id'] = element.attrib['id']
                # Use cleaning function:
                if child.attrib['k'] == 'addr:street':
                    node_tags['value'] = update_name(child.attrib['v'], mapping)
                else:
                    node_tags['value'] = child.attrib['v']
            tags.append(node_tags)

        return {'node': node_attribs, 'node_tags': tags}

    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]

        count = 0
        for child in element:
            way_tag = {}
            way_node = {}

            if child.tag == 'tag':
                if PROBLEMCHARS.match(child.attrib['k']):
                    continue
                elif LOWER_COLON.match(child.attrib['k']):
                    way_tag['id'] = element.attrib['id']
                    way_tag['key'] = child.attrib['k'].split(":", 1)[1]
                    way_tag['type'] = child.attrib['k'].split(":", 1)[0]
                    # Use cleaning function:
                    if child.attrib['k'] == 'addr:street':
                        way_tag['value'] = update_name(child.attrib['v'], mapping)
                    else:
                        way_tag['value'] = child.attrib['v']
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    # Use cleaning function:
                    if child.attrib['k'] == 'addr:street':
                        way_tag['value'] = update_name(child.attrib['v'], mapping)
                    else:
                        way_tag['value'] = child.attrib['v']
                tags.append(way_tag)
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = count
                count += 1
                way_nodes.append(way_node)

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

## Overview of Dataset

I used SQL queries are used to get a statistical overview of the dataset, like: <br>
- size of the file
- number of nodes and ways
- number of unique users
- number of chosen type of nodes, like cafes
- top 5 contributing users
- Popular cuisines

### File sizes

nodes.csv...............................: 38M  
nodes_tags.csv..........................: 1M   
OpenStreetMap.db........................: 56M  
sample.osm..............................: 1M   
ways.csv................................: 3M   
ways_nodes.csv..........................: 13M  
ways_tags.csv...........................: 4M  

### Number of nodes

In [ ]:
sqlite> SELECT COUNT(*) FROM nodes;

492980

### Number of ways

In [ ]:
sqlite> SELECT COUNT(*) FROM ways;

69973

### Number of unique users

In [ ]:
sqlite> SELECT COUNT(DISTINCT(sub.uid))          
   ...> FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) sub;

541

### Number of cafes

In [ ]:
sqlite> SELECT value, COUNT(*)
   ...> FROM (SELECT * FROM nodes_tags as NT UNION ALL
   ...>       SELECT * FROM ways_tags as WT) as sub
   ...> WHERE value = 'cafe';

cafe,185

### Popular cuisines

In [ ]:
sqlite> SELECT NT.value, COUNT(*) as num
   ...> FROM nodes_tags as NT
   ...>      JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') sub
   ...>      ON NT.id = sub.id
   ...> WHERE NT.key = 'cuisine'
   ...> GROUP BY NT.value
   ...> ORDER BY num DESC
   ...> LIMIT 10;

mexican,26 <br>
pizza,26 <br>
thai,24 <br>
japanese,23 <br>
american,15 <br>
italian,15 <br>
indian,14 <br>
chinese,13 <br>
burger,11 <br>
sushi,9

### Top 5 contributing users

In [ ]:
sqlite> SELECT sub.user, COUNT(*) as num
   ...> FROM (SELECT user FROM nodes UNION ALL
   ...>       SELECT user FROM ways) sub
   ...> GROUP BY sub.user
   ...> ORDER BY num DESC
   ...> LIMIT 5;

andygol,290847 <br>
dannykath,82683 <br>
oba510,33779 <br>
dchiles,20177 <br>
RichRico,18537

In [ ]:
sqlite> SELECT SUM(num) as Total
   ...> FROM
   ...> (SELECT sub.user, COUNT(*) as num
   ...>  FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) sub
   ...>        GROUP BY sub.user
   ...>        ORDER BY num DESC) U;

Total contribution by all users = 562953

## Additional Idea about Data

By exploring the dataset, I noticed some percentage statistics: <br>
- top 1 user (andygol) contribution percentage: 51.66%
- top 5 users contribution percentage: 79.23%

We can notice that the top contributing users plays more important roles on the performance of data. I think that could make the data has some bias or mistakes caused by top contributing users. So, if user ID can be displayed and encourage others to make changes on information created by those top contributing users, then this map dataset would be more precise, I hope.

## References

- Udacity
- mablatnik's Github: https://github.com/mablatnik/Wrangle-OpenStreetMap-Data